<img src="../images/cs_logo_pink.PNG" style="float: left; margin: 36px 20px 0 0; height: 60px">

# Capstone Project - Cos Skin <br><i style = "font-size:16px">Your skin but better</i>

## Notebook 7: App Deployment 
Notebook 1: Introduction & Data Collection Part 1 of 3<br>
Notebook 2: Data Collection Part 2 of 3<br>
Notebook 3: Data Collection Part 3 of 3<br>
Notebook 4: EDA & Data Cleaning<br>
Notebook 5: Preprocessing<br>
Notebook 6: Modelling<br>
<b>Notebook 7: App Deployment</b>

In this notebook, I will deploy my recommender under a docker file and upload it to google cloud.<br>
Streamlit will be used to visualize the recommender. 

In [1]:
# import libraries
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
# import datasets
cleanser_pdt_details = pd.read_csv('../data/cleanser_cleaned.csv')
toner_pdt_details = pd.read_csv('../data/toner_cleaned.csv')
day_moisturizer_pdt_details = pd.read_csv('../data/day_moisturizer_cleaned.csv')
night_cream_pdt_details = pd.read_csv('../data/night_cream_cleaned.csv')
sunscreen_pdt_details = pd.read_csv('../data/sunscreen_cleaned.csv')

recommender_cleanser = pd.read_csv('../data/recommender_cleanser.csv', index_col = 0)
recommender_toner = pd.read_csv('../data/recommender_toner.csv', index_col = 0)
recommender_day_moisturizer = pd.read_csv('../data/recommender_day_moisturizer.csv', index_col = 0)
recommender_night_cream = pd.read_csv('../data/recommender_night_cream.csv', index_col = 0)
recommender_sunscreen = pd.read_csv('../data/recommender_sunscreen.csv', index_col = 0)

In [3]:
cleanser_pdt_details.drop(columns = ['skin_type', 'skin_concerns', 'formulation', 'age', 'category'], inplace = True)
toner_pdt_details.drop(columns = ['skin_type', 'skin_concerns', 'formulation', 'age', 'category'], inplace = True)
day_moisturizer_pdt_details.drop(columns = ['skin_type', 'skin_concerns', 'formulation', 'age', 'category'], inplace = True)
night_cream_pdt_details.drop(columns = ['skin_type', 'skin_concerns', 'formulation', 'age', 'category'], inplace = True)
sunscreen_pdt_details.drop(columns = ['skin_type', 'skin_concerns', 'formulation', 'age', 'category'], inplace = True)

In [4]:
pickle.dump(cleanser_pdt_details.to_dict(), open('models/cleanser_pdt_details.pk1', 'wb'))
pickle.dump(toner_pdt_details.to_dict(), open('models/toner_pdt_details.pk1', 'wb'))
pickle.dump(day_moisturizer_pdt_details.to_dict(), open('models/day_moisturizer_pdt_details.pk1', 'wb'))
pickle.dump(night_cream_pdt_details.to_dict(), open('models/night_cream_pdt_details.pk1', 'wb'))
pickle.dump(sunscreen_pdt_details.to_dict(), open('models/sunscreen_pdt_details.pk1', 'wb'))

pickle.dump(recommender_cleanser.to_dict(), open('models/recommender_cleanser.pk1', 'wb'))
pickle.dump(recommender_toner.to_dict(), open('models/recommender_toner.pk1', 'wb'))
pickle.dump(recommender_day_moisturizer.to_dict(), open('models/recommender_day_moisturizer.pk1', 'wb'))
pickle.dump(recommender_night_cream.to_dict(), open('models/recommender_night_cream.pk1', 'wb'))
pickle.dump(recommender_sunscreen.to_dict(), open('models/recommender_sunscreen.pk1', 'wb'))

## Flask API

In [5]:
%%writefile inference.py
from flask import Flask, request
import pandas as pd
import os
import numpy as np 
import random 
from io import StringIO
import pickle

api = Flask('ModelEndpoint')

cleanser_pickle = pickle.load(open('models/cleanser_pdt_details.pk1', 'rb'))
cleanser_pdt_details = pd.DataFrame(cleanser_pickle)
recommender_cleanser_pickle = pickle.load(open('models/recommender_cleanser.pk1', 'rb'))
recommender_cleanser = pd.DataFrame(recommender_cleanser_pickle)

toner_pickle = pickle.load(open('models/toner_pdt_details.pk1', 'rb'))
toner_pdt_details = pd.DataFrame(toner_pickle)
recommender_toner_pickle = pickle.load(open('models/recommender_toner.pk1', 'rb'))
recommender_toner = pd.DataFrame(recommender_toner_pickle)

day_moisturizer_pickle = pickle.load(open('models/day_moisturizer_pdt_details.pk1', 'rb'))
day_moisturizer_pdt_details = pd.DataFrame(day_moisturizer_pickle)
recommender_day_moisturizer_pickle = pickle.load(open('models/recommender_day_moisturizer.pk1', 'rb'))
recommender_day_moisturizer = pd.DataFrame(recommender_day_moisturizer_pickle)

night_cream_pickle = pickle.load(open('models/night_cream_pdt_details.pk1', 'rb'))
night_cream_pdt_details = pd.DataFrame(night_cream_pickle)
recommender_night_cream_pickle = pickle.load(open('models/recommender_night_cream.pk1', 'rb'))
recommender_night_cream = pd.DataFrame(recommender_night_cream_pickle)

sunscreen_pickle = pickle.load(open('models/sunscreen_pdt_details.pk1', 'rb'))
sunscreen_pdt_details = pd.DataFrame(sunscreen_pickle)
recommender_sunscreen_pickle = pickle.load(open('models/recommender_sunscreen.pk1', 'rb'))
recommender_sunscreen = pd.DataFrame(recommender_sunscreen_pickle)

@api.route('/') 
def home(): 
    return {"message": "Hello!", "success": True}, 200

@api.route('/recommendations', methods = ['POST'])
def recommend():
    user_input = request.get_json(force = True)
    user_input = pd.read_json(user_input, lines = True)
    
    # Cleanser recommendation
    cleanser_columns = recommender_cleanser.columns
    cleanser_profile = pd.Series(data = np.zeros(len(cleanser_columns)), index = cleanser_columns) # initialize 0s for all genres to create new user vector using: (https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
    cleanser_profile['Under20'] = user_input['Under20']
    cleanser_profile['20s'] = user_input['20s']
    cleanser_profile['30s'] = user_input['30s']
    cleanser_profile['40s'] = user_input['40s']
    cleanser_profile['50+'] = user_input['50+']
    cleanser_profile['Combination'] = user_input['Combination']
    cleanser_profile['Dry'] = user_input['Dry']
    cleanser_profile['Normal'] = user_input['Normal']
    cleanser_profile['Oily'] = user_input['Oily']
    cleanser_profile['Sensitive'] = user_input['Sensitive']
    cleanser_profile['Ageing'] = user_input['Ageing']
    cleanser_profile['Blackheads'] = user_input['Blackheads']
    cleanser_profile['Blemishes'] = user_input['Blemishes']
    cleanser_profile['DarkCircles'] = user_input['DarkCircles']
    cleanser_profile['Dryness'] = user_input['Dryness']
    cleanser_profile['Dullness'] = user_input['Dullness']
    cleanser_profile['FineLines&Wrinkles'] = user_input['FineLines&Wrinkles']
    cleanser_profile['Firmness&Elasticity'] = user_input['Firmness&Elasticity']
    cleanser_profile['Oiliness'] = user_input['Oiliness']
    cleanser_profile['Pigmentation&DarkSpots'] = user_input['Pigmentation&DarkSpots']
    cleanser_profile['Puffiness'] = user_input['Puffiness']
    cleanser_profile['UnevenSkinTexture'] = user_input['UnevenSkinTexture']
    cleanser_profile['UnevenSkinTone'] = user_input['UnevenSkinTone']
    cleanser_profile['VisiblePores'] = user_input['VisiblePores']
    cleanser_profile['Balm'] = user_input['Balm']
    cleanser_profile['Bar'] = user_input['Bar']
    cleanser_profile['ClayMud'] = user_input['ClayMud']
    cleanser_profile['Cream'] = user_input['Cream']
    cleanser_profile['Foam'] = user_input['Foam']
    cleanser_profile['Gel'] = user_input['Gel']
    cleanser_profile['Liquid'] = user_input['Liquid']
    cleanser_profile['Lotion'] = user_input['Lotion']
    cleanser_profile['Oil'] = user_input['Oil']
    cleanser_profile['Powder'] = user_input['Powder']
    cleanser_profile['Wipe'] = user_input['Wipe']
    cleanser_profile['rating'] = 1.5
    
    cleanser_recommendations = np.dot(recommender_cleanser.values, cleanser_profile.values)
    cleanser_recommendations = pd.Series(cleanser_recommendations, index=recommender_cleanser.index)
    recommended_cleanser = cleanser_recommendations.sort_values(ascending = False).index[0]
    
    cleanser_name = cleanser_pdt_details[cleanser_pdt_details['unique_id'] == recommended_cleanser]['pdt_name'].values[0]
    cleanser_brand = cleanser_pdt_details[cleanser_pdt_details['unique_id'] == recommended_cleanser]['brand'].values[0]
    cleanser_price = cleanser_pdt_details[cleanser_pdt_details['unique_id'] == recommended_cleanser]['price'].values[0].astype(str)
    cleanser_url = cleanser_pdt_details[cleanser_pdt_details['unique_id'] == recommended_cleanser]['pdt_url'].values[0]
    cleanser_image = cleanser_pdt_details[cleanser_pdt_details['unique_id'] == recommended_cleanser]['pdt_images'].values[0]
    
    # Toner recommendation
    toner_columns = recommender_toner.columns
    toner_profile = pd.Series(data = np.zeros(len(toner_columns)), index = toner_columns) # initialize 0s for all genres to create new user vector using: (https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
    toner_profile['Under20'] = user_input['Under20']
    toner_profile['20s'] = user_input['20s']
    toner_profile['30s'] = user_input['30s']
    toner_profile['40s'] = user_input['40s']
    toner_profile['50+'] = user_input['50+']
    toner_profile['Combination'] = user_input['Combination']
    toner_profile['Dry'] = user_input['Dry']
    toner_profile['Normal'] = user_input['Normal']
    toner_profile['Oily'] = user_input['Oily']
    toner_profile['Sensitive'] = user_input['Sensitive']
    toner_profile['Ageing'] = user_input['Ageing']
    toner_profile['Blackheads'] = user_input['Blackheads']
    toner_profile['Blemishes'] = user_input['Blemishes']
    toner_profile['DarkCircles'] = user_input['DarkCircles']
    toner_profile['Dryness'] = user_input['Dryness']
    toner_profile['Dullness'] = user_input['Dullness']
    toner_profile['FineLines&Wrinkles'] = user_input['FineLines&Wrinkles']
    toner_profile['Firmness&Elasticity'] = user_input['Firmness&Elasticity']
    toner_profile['Oiliness'] = user_input['Oiliness']
    toner_profile['Pigmentation&DarkSpots'] = user_input['Pigmentation&DarkSpots']
    toner_profile['Puffiness'] = user_input['Puffiness']
    toner_profile['UnevenSkinTexture'] = user_input['UnevenSkinTexture']
    toner_profile['UnevenSkinTone'] = user_input['UnevenSkinTone']
    toner_profile['VisiblePores'] = user_input['VisiblePores']
    toner_profile['Cream'] = user_input['Cream']
    toner_profile['Gel'] = user_input['Gel']
    toner_profile['Liquid'] = user_input['Liquid']
    toner_profile['Lotion'] = user_input['Lotion']
    toner_profile['Sheet'] = user_input['Sheet']
    toner_profile['Spray'] = user_input['Spray']
    toner_profile['Wipe'] = user_input['Wipe']
    toner_profile['rating'] = 1.5
    
    toner_recommendations = np.dot(recommender_toner.values, toner_profile.values)
    toner_recommendations = pd.Series(toner_recommendations, index=recommender_toner.index)
    recommended_toner = toner_recommendations.sort_values(ascending = False).index[0]
    
    toner_name = toner_pdt_details[toner_pdt_details['unique_id'] == recommended_toner]['pdt_name'].values[0]
    toner_brand = toner_pdt_details[toner_pdt_details['unique_id'] == recommended_toner]['brand'].values[0]
    toner_price = toner_pdt_details[toner_pdt_details['unique_id'] == recommended_toner]['price'].values[0].astype(str)
    toner_url = toner_pdt_details[toner_pdt_details['unique_id'] == recommended_toner]['pdt_url'].values[0]
    toner_image = toner_pdt_details[toner_pdt_details['unique_id'] == recommended_toner]['pdt_images'].values[0]
    
    # Day moisturizer recommendation
    day_moisturizer_columns = recommender_day_moisturizer.columns
    day_moisturizer_profile = pd.Series(data = np.zeros(len(day_moisturizer_columns)), index = day_moisturizer_columns) # initialize 0s for all genres to create new user vector using: (https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
    day_moisturizer_profile['Under20'] = user_input['Under20']
    day_moisturizer_profile['20s'] = user_input['20s']
    day_moisturizer_profile['30s'] = user_input['30s']
    day_moisturizer_profile['40s'] = user_input['40s']
    day_moisturizer_profile['50+'] = user_input['50+']
    day_moisturizer_profile['Combination'] = user_input['Combination']
    day_moisturizer_profile['Dry'] = user_input['Dry']
    day_moisturizer_profile['Normal'] = user_input['Normal']
    day_moisturizer_profile['Oily'] = user_input['Oily']
    day_moisturizer_profile['Sensitive'] = user_input['Sensitive']
    day_moisturizer_profile['Ageing'] = user_input['Ageing']
    day_moisturizer_profile['Blackheads'] = user_input['Blackheads']
    day_moisturizer_profile['Blemishes'] = user_input['Blemishes']
    day_moisturizer_profile['DarkCircles'] = user_input['DarkCircles']
    day_moisturizer_profile['Dryness'] = user_input['Dryness']
    day_moisturizer_profile['Dullness'] = user_input['Dullness']
    day_moisturizer_profile['FineLines&Wrinkles'] = user_input['FineLines&Wrinkles']
    day_moisturizer_profile['Firmness&Elasticity'] = user_input['Firmness&Elasticity']
    day_moisturizer_profile['Oiliness'] = user_input['Oiliness']
    day_moisturizer_profile['Pigmentation&DarkSpots'] = user_input['Pigmentation&DarkSpots']
    day_moisturizer_profile['Puffiness'] = user_input['Puffiness']
    day_moisturizer_profile['UnevenSkinTexture'] = user_input['UnevenSkinTexture']
    day_moisturizer_profile['UnevenSkinTone'] = user_input['UnevenSkinTone']
    day_moisturizer_profile['VisiblePores'] = user_input['VisiblePores']
    day_moisturizer_profile['Balm'] = user_input['Balm']
    day_moisturizer_profile['Cream'] = user_input['Cream']
    day_moisturizer_profile['Gel'] = user_input['Gel']
    day_moisturizer_profile['Liquid'] = user_input['Liquid']
    day_moisturizer_profile['Lotion'] = user_input['Lotion']
    day_moisturizer_profile['Oil'] = user_input['Oil']
    day_moisturizer_profile['Spray'] = user_input['Spray']
    day_moisturizer_profile['rating'] = 1.5
    
    day_moisturizer_recommendations = np.dot(recommender_day_moisturizer.values, day_moisturizer_profile.values)
    day_moisturizer_recommendations = pd.Series(day_moisturizer_recommendations, index=recommender_day_moisturizer.index)
    recommended_day_moisturizer = day_moisturizer_recommendations.sort_values(ascending = False).index[0]
    
    day_moisturizer_name = day_moisturizer_pdt_details[day_moisturizer_pdt_details['unique_id'] == recommended_day_moisturizer]['pdt_name'].values[0]
    day_moisturizer_brand = day_moisturizer_pdt_details[day_moisturizer_pdt_details['unique_id'] == recommended_day_moisturizer]['brand'].values[0]
    day_moisturizer_price = day_moisturizer_pdt_details[day_moisturizer_pdt_details['unique_id'] == recommended_day_moisturizer]['price'].values[0].astype(str)
    day_moisturizer_url = day_moisturizer_pdt_details[day_moisturizer_pdt_details['unique_id'] == recommended_day_moisturizer]['pdt_url'].values[0]
    day_moisturizer_image = day_moisturizer_pdt_details[day_moisturizer_pdt_details['unique_id'] == recommended_day_moisturizer]['pdt_images'].values[0]
    
    # Night cream recommendation
    night_cream_columns = recommender_night_cream.columns
    night_cream_profile = pd.Series(data = np.zeros(len(night_cream_columns)), index = night_cream_columns) # initialize 0s for all genres to create new user vector using: (https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
    night_cream_profile['Under20'] = user_input['Under20']
    night_cream_profile['20s'] = user_input['20s']
    night_cream_profile['30s'] = user_input['30s']
    night_cream_profile['40s'] = user_input['40s']
    night_cream_profile['50+'] = user_input['50+']
    night_cream_profile['Combination'] = user_input['Combination']
    night_cream_profile['Dry'] = user_input['Dry']
    night_cream_profile['Normal'] = user_input['Normal']
    night_cream_profile['Oily'] = user_input['Oily']
    night_cream_profile['Sensitive'] = user_input['Sensitive']
    night_cream_profile['Ageing'] = user_input['Ageing']
    night_cream_profile['Blackheads'] = user_input['Blackheads']
    night_cream_profile['Blemishes'] = user_input['Blemishes']
    night_cream_profile['Dryness'] = user_input['Dryness']
    night_cream_profile['Dullness'] = user_input['Dullness']
    night_cream_profile['FineLines&Wrinkles'] = user_input['FineLines&Wrinkles']
    night_cream_profile['Firmness&Elasticity'] = user_input['Firmness&Elasticity']
    night_cream_profile['Oiliness'] = user_input['Oiliness']
    night_cream_profile['Pigmentation&DarkSpots'] = user_input['Pigmentation&DarkSpots']
    night_cream_profile['Puffiness'] = user_input['Puffiness']
    night_cream_profile['UnevenSkinTexture'] = user_input['UnevenSkinTexture']
    night_cream_profile['UnevenSkinTone'] = user_input['UnevenSkinTone']
    night_cream_profile['VisiblePores'] = user_input['VisiblePores']
    night_cream_profile['Balm'] = user_input['Balm']
    night_cream_profile['Cream'] = user_input['Cream']
    night_cream_profile['Foam'] = user_input['Foam']
    night_cream_profile['Gel'] = user_input['Gel']
    night_cream_profile['Liquid'] = user_input['Liquid']
    night_cream_profile['Lotion'] = user_input['Lotion']
    night_cream_profile['Oil'] = user_input['Oil']
    night_cream_profile['rating'] = 1.5
    
    night_cream_recommendations = np.dot(recommender_night_cream.values, night_cream_profile.values)
    night_cream_recommendations = pd.Series(night_cream_recommendations, index=recommender_night_cream.index)
    recommended_night_cream = night_cream_recommendations.sort_values(ascending = False).index[0]
    
    night_cream_name = night_cream_pdt_details[night_cream_pdt_details['unique_id'] == recommended_night_cream]['pdt_name'].values[0]
    night_cream_brand = night_cream_pdt_details[night_cream_pdt_details['unique_id'] == recommended_night_cream]['brand'].values[0]
    night_cream_price = night_cream_pdt_details[night_cream_pdt_details['unique_id'] == recommended_night_cream]['price'].values[0].astype(str)
    night_cream_url = night_cream_pdt_details[night_cream_pdt_details['unique_id'] == recommended_night_cream]['pdt_url'].values[0]
    night_cream_image = night_cream_pdt_details[night_cream_pdt_details['unique_id'] == recommended_night_cream]['pdt_images'].values[0]
    
    # Sunscreen recommendation
    sunscreen_columns = recommender_sunscreen.columns
    sunscreen_profile = pd.Series(data = np.zeros(len(sunscreen_columns)), index = sunscreen_columns) # initialize 0s for all genres to create new user vector using: (https://numpy.org/doc/stable/reference/generated/numpy.zeros.html)
    sunscreen_profile['Under20'] = user_input['Under20']
    sunscreen_profile['20s'] = user_input['20s']
    sunscreen_profile['30s'] = user_input['30s']
    sunscreen_profile['40s'] = user_input['40s']
    sunscreen_profile['50+'] = user_input['50+']
    sunscreen_profile['Combination'] = user_input['Combination']
    sunscreen_profile['Dry'] = user_input['Dry']
    sunscreen_profile['Normal'] = user_input['Normal']
    sunscreen_profile['Oily'] = user_input['Oily']
    sunscreen_profile['Sensitive'] = user_input['Sensitive']
    sunscreen_profile['Ageing'] = user_input['Ageing']
    sunscreen_profile['Blackheads'] = user_input['Blackheads']
    sunscreen_profile['Blemishes'] = user_input['Blemishes']
    sunscreen_profile['DarkCircles'] = user_input['DarkCircles']
    sunscreen_profile['Dryness'] = user_input['Dryness']
    sunscreen_profile['Dullness'] = user_input['Dullness']
    sunscreen_profile['FineLines&Wrinkles'] = user_input['FineLines&Wrinkles']
    sunscreen_profile['Firmness&Elasticity'] = user_input['Firmness&Elasticity']
    sunscreen_profile['Oiliness'] = user_input['Oiliness']
    sunscreen_profile['Pigmentation&DarkSpots'] = user_input['Pigmentation&DarkSpots']
    sunscreen_profile['UnevenSkinTexture'] = user_input['UnevenSkinTexture']
    sunscreen_profile['UnevenSkinTone'] = user_input['UnevenSkinTone']
    sunscreen_profile['VisiblePores'] = user_input['VisiblePores']
    sunscreen_profile['Balm'] = user_input['Balm']
    sunscreen_profile['Cream'] = user_input['Cream']
    sunscreen_profile['Gel'] = user_input['Gel']
    sunscreen_profile['Liquid'] = user_input['Liquid']
    sunscreen_profile['LoosePowder'] = user_input['LoosePowder']
    sunscreen_profile['Lotion'] = user_input['Lotion']
    sunscreen_profile['Oil'] = user_input['Oil']
    sunscreen_profile['Spray'] = user_input['Spray']
    sunscreen_profile['rating'] = 1.5
    
    sunscreen_recommendations = np.dot(recommender_sunscreen.values, sunscreen_profile.values)
    sunscreen_recommendations = pd.Series(sunscreen_recommendations, index=recommender_sunscreen.index)
    recommended_sunscreen = sunscreen_recommendations.sort_values(ascending = False).index[0]
    
    sunscreen_name = sunscreen_pdt_details[sunscreen_pdt_details['unique_id'] == recommended_sunscreen]['pdt_name'].values[0]
    sunscreen_brand = sunscreen_pdt_details[sunscreen_pdt_details['unique_id'] == recommended_sunscreen]['brand'].values[0]
    sunscreen_price = sunscreen_pdt_details[sunscreen_pdt_details['unique_id'] == recommended_sunscreen]['price'].values[0].astype(str)
    sunscreen_url = sunscreen_pdt_details[sunscreen_pdt_details['unique_id'] == recommended_sunscreen]['pdt_url'].values[0]
    sunscreen_image = sunscreen_pdt_details[sunscreen_pdt_details['unique_id'] == recommended_sunscreen]['pdt_images'].values[0]
    
    return {'cleanser_name': cleanser_name, 'cleanser_brand': cleanser_brand, 'cleanser_price': cleanser_price, 'cleanser_url': cleanser_url, 'cleanser_image': cleanser_image,
            'toner_name':toner_name, 'toner_brand':toner_brand,'toner_price':toner_price, 'toner_url':toner_url, 'toner_image':toner_image,
            'day_moisturizer_name':day_moisturizer_name, 'day_moisturizer_brand':day_moisturizer_brand,'day_moisturizer_price':day_moisturizer_price, 'day_moisturizer_url':day_moisturizer_url, 'day_moisturizer_image':day_moisturizer_image,
            'night_cream_name':night_cream_name, 'night_cream_brand':night_cream_brand,'night_cream_price':night_cream_price, 'night_cream_url':night_cream_url, 'night_cream_image':night_cream_image,
            'sunscreen_name':sunscreen_name, 'sunscreen_brand':sunscreen_brand,'sunscreen_price':sunscreen_price, 'sunscreen_url':sunscreen_url, 'sunscreen_image':sunscreen_image,
           }

if __name__ == '__main__': 
    api.run(host='0.0.0.0', 
            debug=True, 
            port=int(os.environ.get("PORT", 8080))
           ) 

Overwriting inference.py


## Docker

In [6]:
%%writefile Dockerfile
# Use the official lightweight Python image from
# https://hub.docker.com/_/python
FROM python:3.9-slim 
RUN pip install --upgrade pip

# Copy all the files needed for the app to work
COPY inference.py .
COPY requirements.txt .
COPY models/ ./models

# Install all the necessary libraries
RUN pip3 install -r requirements.txt 

# Run the API!
CMD python inference.py

Overwriting Dockerfile


In [7]:
%%writefile requirements.txt
streamlit
flask
lxml==4.9.1

Overwriting requirements.txt


## API Test

In [8]:
import requests, json

user_input = user_input = {'Under20':0, '20s':1, '30s':0, '40s':0, '50+':0,
                           'Combination':0, 'Dry':0, 'Normal':0, 'Oily':1, 'Sensitive':0,
                           'Ageing':3, 'Blackheads':3, 'Blemishes':3, 'DarkCircles':3, 'Dryness':0,  'Dullness':3, 
                           'FineLines&Wrinkles':0, 'Firmness&Elasticity':0, 'Oiliness':3, 'Pigmentation&DarkSpots':0, 
                           'Puffiness':0, 'UnevenSkinTexture':0, 'UnevenSkinTone':0, 'VisiblePores':0,  
                           'Balm':0, 'Bar':0, 'ClayMud':0, 'Cream':0, 'Foam':2,  'Gel':2, 'Liquid':0, 
                           'LoosePowder':0, 'Lotion':0, 'Oil':0, 'Powder':0, 'Sheet':0, 'Spray':0, 'Wipe':0}

api_url = 'https://cosskin-3py4usdxgq-as.a.run.app'
api_route = '/recommendations'

response = requests.post(f'{api_url}{api_route}', json=json.dumps(user_input))
output = response.json()

cleanser_name = output['cleanser_name']
cleanser_brand = output['cleanser_brand']
cleanser_price = output['cleanser_price']
cleanser_url = output['cleanser_url']
cleanser_image = output['cleanser_image']

toner_name = output['toner_name']
toner_brand = output['toner_brand']
toner_price = output['toner_price']
toner_url = output['toner_url']
toner_image = output['toner_image']

day_moisturizer_name = output['day_moisturizer_name']
day_moisturizer_brand = output['day_moisturizer_brand']
day_moisturizer_price = output['day_moisturizer_price']
day_moisturizer_url = output['day_moisturizer_url']
day_moisturizer_image = output['day_moisturizer_image']

night_cream_name = output['night_cream_name']
night_cream_brand = output['night_cream_brand']
night_cream_price = output['night_cream_price']
night_cream_url = output['night_cream_url']
night_cream_image = output['night_cream_image']

sunscreen_name = output['sunscreen_name']
sunscreen_brand = output['sunscreen_brand']
sunscreen_price = output['sunscreen_price']
sunscreen_url = output['sunscreen_url']
sunscreen_image = output['sunscreen_image']

print(f'Cleanser output:\n{cleanser_name}\n{cleanser_brand}\n{cleanser_price}\n{cleanser_url}\n{cleanser_image}\n---')
print(f'Toner output:\n{toner_name}\n{toner_brand}\n{toner_price}\n{toner_url}\n{toner_image}\n---')
print(f'Day Moisturizer output:\n{day_moisturizer_name}\n{day_moisturizer_brand}\n{day_moisturizer_price}\n{day_moisturizer_url}\n{day_moisturizer_image}\n---')
print(f'Night Cream output:\n{night_cream_name}\n{night_cream_brand}\n{night_cream_price}\n{night_cream_url}\n{night_cream_image}\n---')
print(f'Sunscreen output:\n{sunscreen_name}\n{sunscreen_brand}\n{sunscreen_price}\n{sunscreen_url}\n{sunscreen_image}\n')

Cleanser output:
Ceramic Slip Clay Cleanser
SUNDAY RILEY
56
https://www.sephora.sg/products/sunday-riley-ceramic-slip-clay-cleanser
https://image-optimizer-reg.production.sephora-asia.net/images/product_images/closeup_1_Product_200923_20Sunday_20Riley_20Ceramic_20Slip_20Clay_20Cleanser_e1f3d309896a7f7d6b1f38e08ee828529d9ded8a_1541728126.png
---
Toner output:
Ctrl-A Teatreement™ Toner
DR.JART+
29
https://www.sephora.sg/products/dr-jart-ctrl-a-teatreement-toner
https://image-optimizer-reg.production.sephora-asia.net/images/product_images/closeup_1_Product_8809642712867-Dr-Jart-Ctrl-A-Teatreement-Toner-120ml_e408aa5fc4bea297e2efa3ae4f590a31b39f671c_1611050914.png
---
Day Moisturizer output:
UV Expert Youth-Shield™ Aqua Gel SPF50 PA++++
LANCÔME
83
https://www.sephora.sg/products/lancome-uv-expert-xl-shield-fresh-uv-aqua-gel
https://image-optimizer-reg.production.sephora-asia.net/images/product_images/closeup_1_Product_4935421669061-Lancome-UV-Expert-Youth-Shield-Aqua-Gel-SPF50-PA30ml_592d6

## Streamlit

In [9]:
%%writefile streamlit_app.py
import streamlit as st
import requests
import json

st.set_page_config(page_title="Cos Skin!", page_icon="✨", layout="wide" )
st.markdown('<link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-1BmE4kWBq78iYhFldvKuhfTAU6auU8tT94WrHftjDbrCEXSU1oBoqyl2QvZ6jIW3" crossorigin="anonymous"/>', unsafe_allow_html=True)
st.markdown("""
            <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ka7Sk0Gln4gmtz2MlQnikT1wXgYsOg+OMhuP+IlRH9sENBO0LRn5q+8nbTov4+1p" crossorigin="anonymous"></script>
            """, unsafe_allow_html=True)

hide_menu_style = """
        <style>
        #MainMenu {visibility: hidden;}
        </style>
        """
st.markdown(hide_menu_style, unsafe_allow_html=True)

st.markdown("""<nav class="navbar navbar-expand-lg navbar-light fixed-top" style="background-color:#fff1f5">
              <div class="container-fluid">
                <a class="navbar-brand" href="/">CS</a>
                <button class="navbar-toggler" type="button" data-bs-toggle="collaspe" data-bs-target="#navbarSupportedContent" aria-controls="navbarSupportedContent" aria-expanded="false" aria-label="Toggle navigation">
                  <span class="navbar-toggler-icon"></span>
                </button>
                <div class="collapse navbar-collapse" id="navbarSupportedContent">
                  <ul class="navbar-nav">
                      <li class="nav-item">
                        <a class="nav-link" href="https://www.linkedin.com/in/clare-seah/" title="Clare's LinkedIn Profile">LinkedIn</a>
                      </li>
                      <li class="nav-item">
                        <a class="nav-link" href="https://github.com/clareseah" title="Clare's GitHub">GitHub</a>
                      </li>
                      <li class="nav-item">
                        <a class="nav-link" href="https://clare-seah.netlify.app/contact.html" title="Contact Clare Seah">Contact</a>
                      </li>
                  </ul>
                </div>
              </div>
            </nav>""", unsafe_allow_html=True)

st.write("""<h2>Welcome to Cos Skin! ✨</h2>
<h5><i>One-stop shop for skincare product recommendations JUST FOR YOU!</i></h5>
<h5>Obtain a personalized 4-step skincare routine by filling up the questionnaire below 👇🏻 </h5>""", unsafe_allow_html=True)

skin_age = st.selectbox('Select your age range:', ('Choose an option','20s', '30s', '40s', '50+', 'Under 20'))
skin_type = st.selectbox('Select your skin type:', ('Choose an option', 'Combination', 'Dry', 'Normal', 'Oily', 'Sensitive'))
skin_concerns = st.multiselect('What are your skin concerns?', ['Ageing', 'Blackheads', 'Blemishes', 'Dark Circles', 'Dryness', 'Dullness', 'Fine Lines & Wrinkles', 'Firmness & Elasticity', 'Oiliness', 'Pigmentation & Dark Spots', 'Puffiness', 'Uneven Skin Texture', 'Uneven Skin Tone', 'Visible Pores'])
formulation = st.multiselect('Do you have any formulation preferences?', ['Balm', 'Bar', 'Clay/Mud', 'Cream', 'Foam', 'Gel', 'Loose Powder', 'Liquid', 'Lotion', 'Oil', 'Powder', 'Sheet', 'Spray', 'Wipe'])

if skin_age == 'Under 20':
    age_under20 = 1
    age_20s = 0
    age_30s = 0
    age_40s = 0
    age_50s = 0
elif skin_age == '20s': 
    age_under20 = 0
    age_20s = 1
    age_30s = 0
    age_40s = 0
    age_50s = 0
elif skin_age == '30s':
    age_under20 = 0
    age_20s = 0
    age_30s = 1
    age_40s = 0
    age_50s = 0
elif skin_age == '40s':
    age_under20 = 0
    age_20s = 0
    age_30s = 0
    age_40s = 1
    age_50s = 0
elif skin_age == '50s':
    age_under20 = 0
    age_20s = 0
    age_30s = 0
    age_40s = 0
    age_50s = 1

if skin_type == 'Combination':
    type_combi = 1
    type_dry = 0
    type_normal = 0
    type_oily = 0
    type_sensitive = 0
elif skin_type == 'Dry':
    type_combi = 0
    type_dry = 1
    type_normal = 0
    type_oily = 0
    type_sensitive = 0
elif skin_type == 'Normal':
    type_combi = 0
    type_dry = 0
    type_normal = 1
    type_oily = 0
    type_sensitive = 0
elif skin_type == 'Oily':
    type_combi = 0
    type_dry = 0
    type_normal = 0
    type_oily = 1
    type_sensitive = 0
elif skin_type == 'Sensitive':
    type_combi = 0
    type_dry = 0
    type_normal = 0
    type_oily = 0
    type_sensitive = 1

if 'Ageing' in skin_concerns:
    concerns_ageing = 3
else:
    concerns_ageing = 0
if 'Blackheads' in skin_concerns:
    concerns_blackheads = 3
else:
    concerns_blackheads = 0
if 'Blemishes' in skin_concerns:
    concerns_blemishes = 3
else:
    concerns_blemishes = 0
if 'Dark Circles' in skin_concerns:
    concerns_darkcircles = 3
else:
    concerns_darkcircles = 0
if 'Dryness' in skin_concerns:
    concerns_dryness = 3
else:
    concerns_dryness = 0
if 'Dullness' in skin_concerns:
    concerns_dullness = 3
else:
    concerns_dullness = 0
if 'Fine Lines & Wrinkles' in skin_concerns:
    concerns_finelines_wrinkles = 3
else: 
    concerns_finelines_wrinkles = 0
if 'Firmness & Elasticity' in skin_concerns:
    concerns_firmness_elasticity = 3
else:
    concerns_firmness_elasticity = 0
if 'Oiliness' in skin_concerns:
    concerns_oiliness = 3
else: 
    concerns_oiliness = 0
if 'Pigmentation & Dark Spots' in skin_concerns:
    concerns_pigmentation_darkspots = 3
else:
    concerns_pigmentation_darkspots = 0
if 'Puffiness' in skin_concerns:
    concerns_puffiness = 3
else: 
    concerns_puffiness = 0
if 'Uneven Skin Texture' in skin_concerns:
    concerns_uneven_skin_texture = 3
else: 
    concerns_uneven_skin_texture = 0
if 'Uneven Skin Tone' in skin_concerns:
    concerns_uneven_skin_tone = 3
else: 
    concerns_uneven_skin_tone = 0
if 'Visible Pores' in skin_concerns:
    concerns_visible_pores = 3
else:
    concerns_visible_pores = 0
    
if 'Balm' in formulation:
    formula_balm = 2
else:
    formula_balm = 0
if 'Bar' in formulation:
    formula_bar = 2
else: 
    formula_bar = 0
if 'Clay/Mud' in formulation:
    formula_clay_mud = 2
else: 
    formula_clay_mud = 0
if 'Cream' in formulation:
    formula_cream = 2
else:
    formula_cream = 0
if 'Foam' in formulation:
    formula_foam = 2
else:
    formula_foam = 0
if 'Gel' in formulation:
    formula_gel = 2
else:
    formula_gel = 0
if 'Liquid' in formulation:
    formula_liquid = 2
else:
    formula_liquid = 0
if 'Loose Powder' in formulation:
    formulation_loose_powder = 2
else:
    formulation_loose_powder = 0
if 'Lotion' in formulation:
    formula_lotion = 2
else:
    formula_lotion = 0
if 'Oil' in formulation:
    formula_oil = 2
else:
    formula_oil = 0
if 'Powder' in formulation:
    formula_powder = 2
else:
    formula_powder = 0
if 'Sheet' in formulation:
    formula_sheet = 2
else: 
    formula_sheet = 0
if 'Spray' in formulation:
    formula_spray = 2
else:
    formula_spray = 0
if 'Wipe' in formulation:
    formula_wipe = 2
else:
    formula_wipe = 0   
    
if st.button('Show Recommendation'):
    
    user_input = {'Under20':age_under20, '20s':age_20s, '30s':age_30s, '40s':age_40s, '50+':age_50s, 
                  'Combination':type_combi, 'Dry':type_dry, 'Normal':type_normal, 'Oily':type_oily, 'Sensitive':type_sensitive, 
                  'Ageing':concerns_ageing, 'Blackheads':concerns_blackheads, 'Blemishes':concerns_blemishes, 'DarkCircles':concerns_darkcircles, 
                  'Dryness':concerns_dryness,  'Dullness':concerns_dullness, 'FineLines&Wrinkles':concerns_finelines_wrinkles, 
                  'Firmness&Elasticity':concerns_firmness_elasticity, 'Oiliness':concerns_oiliness, 
                  'Pigmentation&DarkSpots':concerns_pigmentation_darkspots, 'Puffiness':concerns_puffiness, 
                  'UnevenSkinTexture':concerns_uneven_skin_texture, 'UnevenSkinTone':concerns_uneven_skin_tone, 'VisiblePores':concerns_visible_pores,  
                  'Balm':formula_balm, 'Bar':formula_bar, 'ClayMud':formula_clay_mud, 'Cream':formula_cream, 'Foam':formula_foam,  
                  'Gel':formula_gel, 'Liquid':formula_liquid, 'LoosePowder':formulation_loose_powder, 'Lotion':formula_lotion, 'Oil':formula_oil, 
                  'Powder':formula_powder, 'Sheet':formula_sheet, 'Spray':formula_spray, 'Wipe':formula_wipe}
    
    api_url = 'https://capstone-2h5cv6z6ba-as.a.run.app'
    api_route = '/recommendations'

    response = requests.post(f'{api_url}{api_route}', json=json.dumps(user_input)) # json.dumps() converts dict to JSON
    output = response.json()
 
    cleanser_name = output['cleanser_name']
    cleanser_brand = output['cleanser_brand']
    cleanser_price = output['cleanser_price']
    cleanser_url = output['cleanser_url']
    cleanser_image = output['cleanser_image']

    toner_name = output['toner_name']
    toner_brand = output['toner_brand']
    toner_price = output['toner_price']
    toner_url = output['toner_url']
    toner_image = output['toner_image']

    day_moisturizer_name = output['day_moisturizer_name']
    day_moisturizer_brand = output['day_moisturizer_brand']
    day_moisturizer_price = output['day_moisturizer_price']
    day_moisturizer_url = output['day_moisturizer_url']
    day_moisturizer_image = output['day_moisturizer_image']

    night_cream_name = output['night_cream_name']
    night_cream_brand = output['night_cream_brand']
    night_cream_price = output['night_cream_price']
    night_cream_url = output['night_cream_url']
    night_cream_image = output['night_cream_image']

    sunscreen_name = output['sunscreen_name']
    sunscreen_brand = output['sunscreen_brand']
    sunscreen_price = output['sunscreen_price']
    sunscreen_url = output['sunscreen_url']
    sunscreen_image = output['sunscreen_image']
    
    st.markdown("<br>", unsafe_allow_html = True)
    
    col1, col2, col3, col4, col5 = st.columns(5, gap = "medium")
    
    with col1:
        st.write('<p class = "category">Cleanser</p>', unsafe_allow_html=True)
        st.markdown(f"[![{cleanser_name}]({cleanser_image})]({cleanser_url})")
        st.write(f"""<div class ="pdt-info">
        <a href="{cleanser_url}" target="_blank" class = "brand">{cleanser_brand}</a><br>
        <a href="{cleanser_url}" target="_blank" class = "pdt-name">{cleanser_name}</a><br>
        <a href="{cleanser_url}" target="_blank" class = "pdt-price">${cleanser_price}</a>
        </div>
        """, unsafe_allow_html=True)
        
    with col2:
        st.write('<p class = "category">Toner</p>', unsafe_allow_html=True)
        st.markdown(f"[![{toner_name}]({toner_image})]({toner_url})")
        st.write(f"""<div class ="pdt-info">
        <a href="{toner_url}" target="_blank" class = "brand">{toner_brand}</a><br>
        <a href="{toner_url}" target="_blank" class = "pdt-name">{toner_name}</a><br>
        <a href="{toner_url}" target="_blank" class = "pdt-price">${toner_price}</a>
        </div>
        """, unsafe_allow_html=True)

    with col3:
        st.write('<p class = "category">Day Moisturizer</p>', unsafe_allow_html=True)
        st.markdown(f"[![{day_moisturizer_name}]({day_moisturizer_image})]({day_moisturizer_url})")
        st.write(f"""<div class ="pdt-info">
        <a href="{day_moisturizer_url}" target="_blank" class = "brand">{day_moisturizer_brand}</a><br>
        <a href="{day_moisturizer_url}" target="_blank" class = "pdt-name">{day_moisturizer_name}</a><br>
        <a href="{day_moisturizer_url}" target="_blank" class = "pdt-price">${day_moisturizer_price}</a>
        </div>
        """, unsafe_allow_html=True)

    with col4:
        st.write('<p class = "category">Night Cream</p>', unsafe_allow_html=True)
        st.markdown(f"[![{night_cream_name}]({night_cream_image})]({night_cream_url})")
        st.write(f"""<div class ="pdt-info">
        <a href="{night_cream_url}" target="_blank" class = "brand">{night_cream_brand}</a><br>
        <a href="{night_cream_url}" target="_blank" class = "pdt-name">{night_cream_name}</a><br>
        <a href="{night_cream_url}" target="_blank" class = "pdt-price">${night_cream_price}</a>
        </div>
        """, unsafe_allow_html=True)

    with col5:
        st.write('<p class = "category">Sunscreen</p>', unsafe_allow_html=True)
        st.markdown(f"[![{sunscreen_name}]({sunscreen_image})]({sunscreen_url})")
        st.write(f"""<div class ="pdt-info">
        <a href="{sunscreen_url}" target="_blank" class = "brand">{sunscreen_brand}</a><br>
        <a href="{sunscreen_url}" target="_blank" class = "pdt-name">{sunscreen_name}</a><br>
        <a href="{sunscreen_url}" target="_blank" class = "pdt-price">${sunscreen_price}</a>
        </div>
        """, unsafe_allow_html=True)


write_footer = """
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.0.0/css/all.min.css">  
<br><hr>
<div style="text-align:center">
<p class = "footer">
  <a href="https://github.com/Clarefairy/shecodes-responsive-portfolio" target="_blank" class = "footer" title="Visit open-source code for this website">
  Open-source code</a> by Clare Seah
</p>
<a href="mailto:clareseah@gmail.com" target="_blank" class = "footer" title="Send Clare an email"><i class="far fa-envelope"></i></a>
<a href="https://www.linkedin.com/in/clare-seah/" target="_blank" class = "footer" title="Visit Clare's LinkedIn profile"><i class="fab fa-linkedin"></i></a>
<a href="https://github.com/clareseah" target="_blank" class = "footer" title="Visit Clare's github profile"><i class="fab fa-github-square"></i></a>
</div>
"""        
        
st.write(write_footer, unsafe_allow_html=True)
        
        
def add_bg_from_url():
    st.markdown(
         f"""
         <style>
         .stApp {{
             background-color:  #fff;

         }}
         
        .css-ffhzg2, css-fg4pbf {{
            font-family: CenturyGothic, Helvetica, Arial;
            color: #4F5B67;
        }}
         
         [data-testid = "stHeader"] {{
              background-color: rgba(0,0,0,0);
          }}         
         
         .navbar-brand {{
             font-family: monospace;
             border: 1px solid black;
             padding: 4px 5px;
             margin-left: 10px;
         }}
         
         .nav-link:hover {{
             text-decoration:none
         }}
         
        .css-10trblm, .css-k3w14i {{
            color:  #4F5B67;
        }}
        
        .css-81oif8 {{
            font-family: CenturyGothic, Helvetica, Arial;
        }}
        
        .st-bx, .st-by, .st-bw, .st-d8, .st-fg, .st-fh, .st-d7, .st-d9 {{
            border-color: pink;
            color: #4F5B67;
            background-color: white;
        }}
        
        .st-bt, .st-ff, .st-cs {{
            color: #4F5B67;
        }}
        
        .st-e0, .st-e8 {{
            background-color: pink;
        }}
        
        .st-dz, .st-e7, .st-br {{
            color: white;
        }}
        
        .st-dh, .st-di {{
            background-color: white;
        }}
        
        .st-bs {{
            color: #4f5b67;
        }}
        
        .css-8ojfln {{
            color: #4f5b67;
        }}
        
        .css-c2zpwa, .css-c2zpwa:hover {{
            background: rgb(240, 242, 246);
        
        }}
        
        .css-af4qln h2 {{
            padding: 10px 0 0 0;
        }}
        
        .css-af4qln h5 {{
            padding: 0 0 8px 0;
        }}
        
        .css-1offfwp p.category, .css-1fv8s86 p.category {{
            text-align: center;
            text-transform: uppercase;
            font-size: 18px;
            font-weight: 700;
            
        }}
        
        .css-1offfwp a.brand, .css-1fv8s86 a.brand {{
            color: #4F5B67;
            text-decoration: none;
            line-height: 18px;        
            font-weight: 700;
            text-transform: uppercase;
        }}
        
        .css-1offfwp a.pdt-name, .css-1fv8s86 a.pdt-name {{
            color: #4F5B67;
            text-decoration: none;
            line-height: 18px;        
        }}
        
        .css-1offfwp div.pdt-info, .css-1fv8s86 div.pdt-info {{
            text-align: center;
            margin-bottom: 10px;
        
        }}
        
        .css-1offfwp a.pdt-price, .css-1fv8s86 a.pdt-price {{
            color: #4F5B67;
            text-decoration: none;
            line-height: 18px;        
            font-weight: 700;
        }}
        
        .css-1offfwp a.brand:hover, .css-1offfwp a.pdt-name:hover, .css-1offfwp a.pdt-price:hover, .css-1fv8s86 a.brand:hover, .css-1fv8s86 a.pdt-name:hover, .css-1fv8s86 a.pdt-price:hover {{ 
            color: #C7D0D7;
            transition: all 200ms ease-in-out;
        }}
        
        .css-1offfwp p, .css-1fv8s86 p {{
            color: #4F5B67;
        }}
        
        .css-1offfwp p.pdt-name, .css-1fv8s86 p.pdt-name {{
            line-height: 18px;
            word-wrap: break-word;
            overflow-wrap: break-word;
            margin-bottom: 1px;           
        }}
        
        .css-18e3th9 {{
            padding-top: 2rem;
        }}
        
        .css-vhjbnf, .css-1qz96h7 {{
            padding: 12px;
            border: 1px solid grey;
            border-radius: 25px;
            box-shadow: 0 .5rem 1rem rgba(0,0,0,.15);
        }}
        
        .css-1ghk2ip, .css-v4g7v5 {{
            text-align:center;
        }}
        
        img {{
            margin: auto;
            display: block;
            width: 180px;
            
        }}
       
        .css-1x8cf1d, .css-5uatcg {{
            border: 1px solid #f65282;
            color: #f65282; 
            background-color: rgb(255, 255, 255);
        }}
        
        .css-1x8cf1d:hover, .css-5uatcg:hover {{
            border: 1px solid rgba(254, 181, 200);
            color: rgba(254, 181, 200);
        }}        
                      
        .css-1offfwp p.footer, .css-1fv8s86 p.footer {{
            font-size: 16px;
            margin-top: auto 10px;
        }}
        
        .css-1offfwp i, .css-1fv8s86 i{{
            font-size: 22px;
            color: #f65282;
            padding: 0 10px 0 0;
            text-decoration: none;
        }}
        
        .css-1offfwp i:hover, .css-1fv8s86 i:hover {{
            color: #ffc4c0;
            transition: all 200ms ease-in-out;
        }}
        
        .css-1offfwp a.footer, .css-1fv8s86 a.footer {{
            color: #f65282;
            text-decoration: none;
        }}

        .css-1offfwp a:hover.footer, .css-1fv8s86 a:hover {{
            color: #ffc4c0;
            transition: all 200ms ease-in-out;
        }}
        
         </style>
         """,
         unsafe_allow_html=True
     )

add_bg_from_url() 

Overwriting streamlit_app.py


## Conclusion

By inputting your basic user information (skin type and user's age) and skincare preferences (skin concerns and formulation), the recommender is able to recommend a list of products that form a simple 4-step skincare routine. This recommender will aid new users with decision fatigue and serve as a stepping stone for them to start a skincare routine. For skincare enthusiasts, this recommender can aid in recommending alternative products to broaden their choices (if needed).

## Limitations
- As the data collected is from Sephora's website, the tags are dependent on the company as well. It is likely that they will tag products with as many filter tags as possible to increase appearance of the products when customers use filter options. Hence accuracy of filters tagged cannot be assured.
- User reviews and ratings include those from Sephora US as well. Since Sephora US has a larger and more active customer base, the ratings maybe influenced by US customers. Due to the difference in climate and environments, as well as skin types, the user ratings may not be 100% applicable to customers in Singapore. 


## Future Works
- Add more skincare categories (Mask, Serum, Eye cream etc)
    - Allow users to determine the categories they want recommendations for to create their own skincare routine
- On top of filter tags by Sephora, look into product ingredients to determine the use and effective of products
- Add in User-based recommendation with implicit data and reviews of all attractions